In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from scipy.interpolate import interp1d
import glob
import os
import string
from PIL import Image

In [2]:
def read_density(cont, Nx, Nz):
    '''
    Read density data from density_step.txt to extract interfaces
    '''

    Rho = np.loadtxt("density_"+str(cont)+".txt",skiprows=2, unpack=True, comments="P")
    Rho = np.reshape(Rho, (Nz, Nx))

    return Rho

def extract_interface(z, Z, Nx, Rhoi, rho):
    '''
    Extract interface according to a given density
    '''
    topo_aux = []

    for j in np.arange(Nx):
        topoi = interp1d(z, Rhoi[:,j]) #return a "function" of interpolation to apply in other array
        idx = (np.abs(topoi(Z)-rho)).argmin()
        topo = Z[idx]
        topo_aux = np.append(topo_aux, topo)

    return topo_aux

def extract_topography(step):
    # Read surface topography
    topo= np.loadtxt(f"sp_surface_global_{step}.txt", skiprows=2, unpack=True, comments="P")
    topo = topo/1000
    return topo


In [3]:
with open("param.txt", "r") as f:
    line = f.readline()
    line = line.split()
    Nx = int(line[2])
    line = f.readline()
    line = line.split()
    Nz = int(line[2])
    line = f.readline()
    line = line.split()
    Lx = float(line[2])
    line = f.readline()
    line = line.split()
    Lz = float(line[2])

data = [Nx, Nz, Lx, Lz]

xi = np.linspace(0, Lx / 1e3, Nx)
zi = np.linspace(-Lz / 1e3, 0, Nz)

xx, zz = np.meshgrid(xi, zi)

thickness_air = 40.0

In [4]:
# Initial, final and step size to plot the graphs

step_initial = 0
step_final = 10000

d_step = 20

In [ ]:
t, vel, top, stop = [], [], [], [] # surface topography

for cont in range(step_initial, step_final + d_step, d_step): 
    # Read time
    time = np.loadtxt("time_" + str(cont) + ".txt", dtype="str")
    time = time[:, 2:]
    time = time.astype("float")
    time = time[0]
    
    t = np.append(t, time)
    
    # Read velocity
    veloc = np.loadtxt(
        "velocity_" + str(cont) + ".txt",
        delimiter=" ",
        comments="P",
        skiprows=2,
    )
    velocz = veloc[1::2]
    #velocz[np.abs(velocz) < 1.0e-200] = 0
    velocz = np.reshape(velocz, (Nx, Nz), order="F")
    velocz = np.transpose(velocz)
    vzmean = np.mean(velocz[80:200,730:800])
    
    vel = np.append(vel, vzmean)
    
    # Calculate topography
    Nx, Nz, Lx, Lz = data
    
    z_mean = 40.0 #np.mean(topo[condx])
    
    Rhoi = read_density(cont, Nx, Nz)
    
    ##Extract layer topography
    z = np.linspace(Lz/1000.0, 0, Nz)
    Z = np.linspace(Lz/1000.0, 0, 8001) #zi
    xi = np.linspace(0, Lx/1000, Nx)

    topo_interface = extract_interface(z, Z, Nx, Rhoi, 300.) #200 kg/m3 = air/crust interface
    topo_interface -= np.abs(z_mean) #h_air
    condx = (xi >= 200) & (xi <= 300)
    mean = np.mean(topo_interface[condx])
    topo_interface = topo_interface + np.abs(mean)
    topo_interface = -1.0*topo_interface
    
    ### checar onde que vamos calcular a media
    topo = extract_topography(cont)
    topo -= z_mean #h_air
    condx = (xi >= 200) & (xi <= 300)
    mean = np.mean(topo[condx])
    topo += np.abs(mean)
    
    # ax1.plot(x, topo, color="b")
    
#     topo_lump = np.mean(topo[1100:1320]) # 550 660
    topo_lump = np.mean(topo_interface[550:660])
    topo_topo = np.mean(topo[550:660])
    
    top = np.append(top, topo_lump)
    stop = np.append(stop, topo_topo)
    
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)
fig.suptitle('Correlation between topography and EDC - 26nk')
ax1.plot(t/1e6,vel*(60*60*24*365)*1e2,'r')
ax1.grid('-k', alpha=0.7)
ax1.set_ylabel('Velocity [cm/yr]')
ax1.tick_params(axis='y')
ax1.tick_params(bottom = False)

ax2.plot(t/1e6,top, label='densidade')
ax2.plot(t/1e6,stop, label='surface', color='orange')
ax2.grid('-k', alpha=0.7)
ax2.set_xlim(t[0]/1e6, t[-1]/1e6)
ax2.set_ylabel("Depth [km]")
ax2.legend(loc='upper right')

ax3.plot(t/1e6, stop, color = 'orange')
ax3.grid('-k', alpha=0.7)
ax3.set_ylabel("Depth [km]")
ax3.set_xlabel("Time [Myr]")
plt.savefig('topo26nk', format='png')